In [1]:
from os.path import join, dirname, basename, splitext, abspath
from glob import glob
import pandas as pd
import yaml
import platform
from IPython.display import display
from IPython.core.display import display, HTML
import platform
from util.loader import load_raw_data, convert_data
display(HTML("<style>.container { width:100% !important; }</style>"))

this_folder = dirname(__file__) if '__file__' in globals() else abspath('')
root_folder = dirname(dirname(this_folder))
def get_test_result_folder(testname = 'baseline'):
    return join(root_folder, 'test', 'testresults', 'XPS-15-9560', testname, 'CUTEst')

def color_negative_red(val):
    color = 'red' if val < 0 else 'black'
    return f'color: {color}'
def color_negative_red_positive_green(val):
    if val > 0:
        return 'color: green'
    elif val < 0:
        return 'color: red'
    else:
        return ''

In [2]:
base_folder = get_test_result_folder('panoc-lbfgs-11-4')
new_folder = get_test_result_folder('panoc-2nd-lbfgs')

In [3]:
base_raw = load_raw_data(base_folder)
new_raw = load_raw_data(new_folder)

In [4]:
base_df = convert_data(base_raw)
new_df = convert_data(new_raw)

In [5]:
pd.set_option('display.max_rows', None)
new_df.style.format('{:.2e}', subset=(new_df.dtypes == float)) \
            .format('{:.8e}', subset=['f'])

,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,f evaluations,grad_f evaluations,g evaluations,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,,,
3PK,Converged,1.54e+00,112601,116,111,0,20,1.72012026e+00,9.55e-06,0.00e+00,226450,338031,339147,338031,7,0,0,0.00e+00,3.69e+02,0.00e+00,30,0,30
A4X12,MaxTime,3.09e+01,19000,21,19,0,18,-5.76565509e-01,1.54e+00,3.07e-01,40471,58184,59475,58184,0,0,0,1.96e+01,1.53e+00,5.34e-01,130,385,30
ACOPP14,MaxTime,3.02e+01,145147,148,141,0,20,2.37883313e+03,2.35e-04,1.24e-01,669742,618279,815017,618279,12098,0,0,4.90e+02,4.12e+00,1.05e+04,38,68,48
ACOPP30,MaxTime,3.01e+01,79253,210,56,0,20,2.40481534e+00,2.54e-05,7.46e-02,321670,315454,401113,315454,5641,0,0,6.18e+01,5.69e+00,6.91e+02,72,142,84
ACOPR14,MaxTime,3.01e+01,116305,144,89,0,20,0.00000000e+00,4.00e-05,1.87e-01,712213,579640,828642,579640,16152,0,0,1.37e+02,3.97e+00,3.07e+03,38,82,20
AIRCRFTA,Converged,1.42e-03,178,6,0,0,0,0.00000000e+00,4.48e-06,4.42e-07,386,542,571,542,0,0,0,2.31e+02,1.59e-01,1.65e-06,8,5,6
AIRPORT,Converged,6.08e-01,908,10,0,0,0,4.79526657e+04,9.58e-06,5.26e-06,3560,3557,4478,3557,73,0,0,8.67e+05,3.43e+01,5.03e+03,84,42,168
ALLINITA,Converged,2.24e-03,146,10,0,0,0,3.32486378e+01,3.66e-11,7.12e-06,352,466,508,466,0,0,0,2.66e+08,2.25e+00,3.51e+03,4,4,5
ALLINITC,Converged,2.53e-03,195,10,0,0,0,3.04420769e+01,6.90e-06,3.82e-06,518,620,723,620,2,0,0,1.00e+09,2.29e+00,7.11e+03,4,1,5


In [6]:
def df_stats(df):
    conv = df['status'].value_counts()['Converged']
    tot = df['status'].count()
    tot_time = df['time'].sum()
    conv_time = df.where(df['status'] == 'Converged')['time'].sum()
    print(f'Converged:      {conv}/{tot} = {100*conv/tot:.02f}%')
    print(f'Total time:     {tot_time:.03f}s')
    print(f'Converged time: {conv_time:.03f}s')

In [7]:
print('Baseline\n---\n')
df_stats(base_df)
print('\n')
print('New test\n---\n')
df_stats(new_df)
print('\n')

Baseline
---

Converged:      152/219 = 69.41%
Total time:     1001.530s
Converged time: 78.068s


New test
---

Converged:      152/219 = 69.41%
Total time:     1230.631s
Converged time: 140.889s




In [8]:
def compare_results(a, b, columns):
#     res = pd.DataFrame()
#     for i, df in enumerate(dfs):
#         res[f'{column} {i}'] = df[column]
#     return res
    res = a[columns].join(b[columns], lsuffix=' 0', rsuffix=' 1')
    return res

In [9]:
cmp = compare_results(base_df, new_df, ['status', 'f', 'ε', 'δ']) #  'inner iterations', 'outer iterations',
cmp['f imprv'] = cmp['f 0'] - cmp['f 1']
cmp['rel f imprv'] = 100 * cmp['f imprv'] / abs(cmp['f 0'])
cmp.sort_values('rel f imprv') \
   .style.applymap(color_negative_red_positive_green, subset=['f imprv', 'rel f imprv']) \
         .format('{:.2e}', subset=(cmp.dtypes == float)) \
         .format('{:.8e}', subset=['f 0', 'f 1']) \
         .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), cmp.columns)))

,status 0,f 0,ε 0,δ 0,status 1,f 1,ε 1,δ 1,f imprv,rel f imprv
name,,,,,,,,,,
YORKNET,MaxIter,-1.58689860e+01,1.53e-02,1.15e+01,MaxTime,1.46646773e+04,6.68e-02,3.31e+01,-1.47e+04,-92510.93%
SMMPSF,MaxTime,9.80710691e+03,5.20e-01,3.11e+02,MaxTime,8.51664417e+05,1.22e+00,8.55e+01,-8.42e+05,-8584.16%
SMBANK,Converged,-7.12929200e+06,5.03e-06,2.65e-06,MaxIter,1.45906510e+07,4.69e-06,1.59e+06,-2.17e+07,-304.66%
HS25,Converged,8.81555604e-11,4.44e-06,0.00e+00,Converged,3.46311609e-10,7.42e-06,0.00e+00,-2.58e-10,-292.84%
AVION2,MaxIter,1.55800065e+05,2.49e+01,2.24e+03,MaxIter,3.46872535e+05,3.05e+02,2.88e+04,-1.91e+05,-122.64%
HELSBY,MaxTime,2.74535309e+01,1.33e-01,5.12e-02,MaxTime,5.76493458e+01,2.49e-01,2.46e-03,-3.02e+01,-109.99%
BATCH,MaxIter,1.14997087e+05,6.27e-04,2.02e+00,MaxIter,2.31790805e+05,6.14e-01,2.14e-01,-1.17e+05,-101.56%
A4X12,MaxTime,-7.80200935e+02,1.42e-01,5.63e+00,MaxTime,-5.76565509e-01,1.54e+00,3.07e-01,-7.80e+02,-99.93%
MESH,MaxIter,-4.19224156e+10,1.65e+06,3.15e+03,MaxIter,-5.54894305e+07,3.65e+05,3.24e+02,-4.19e+10,-99.87%


In [10]:
worse_f = cmp[(cmp['f imprv'] < 0) & (cmp['status 1'] == 'Converged')]
better_f = cmp[(cmp['f imprv'] > 0) & (cmp['status 1'] == 'Converged')]
tol = 1e-5
really_worse_f = worse_f[abs(worse_f['rel f imprv']) > 100 * tol]
really_better_f = better_f[abs(better_f['rel f imprv']) > 100 * tol]
print(f'{len(worse_f)} tests got worse results')
print(f'{len(really_worse_f)} tests got significantly worse')
print(f'{len(better_f)} tests got better results')
print(f'{len(really_better_f)} tests got significantly better')

print('\nSignificantly worse tests:')
really_worse_f.sort_values('rel f imprv') \
              .style.applymap(color_negative_red_positive_green, subset=['f imprv', 'rel f imprv']) \
                    .format('{:.2e}', subset=(really_worse_f.dtypes == float)) \
                    .format('{:.8e}', subset=['f 0', 'f 1']) \
                    .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), really_worse_f.columns)))

62 tests got worse results
8 tests got significantly worse
59 tests got better results
5 tests got significantly better

Significantly worse tests:


,status 0,f 0,ε 0,δ 0,status 1,f 1,ε 1,δ 1,f imprv,rel f imprv
name,,,,,,,,,,
HS25,Converged,8.81555604e-11,4.44e-06,0.00e+00,Converged,3.46311609e-10,7.42e-06,0.00e+00,-2.58e-10,-292.84%
MATRIX2,Converged,8.47159631e-12,6.71e-06,1.02e-10,Converged,1.03159286e-11,3.32e-06,5.14e-12,-1.84e-12,-21.77%
KISSING2,Converged,5.26549143e+00,9.73e-06,6.88e-07,Converged,6.21607759e+00,9.39e-06,9.88e-07,-9.51e-01,-18.05%
DECONVU,Converged,1.14971862e-07,4.61e-06,0.00e+00,Converged,1.22659787e-07,8.15e-06,0.00e+00,-7.69e-09,-6.69%
DECONVC,Converged,1.16796572e-08,8.04e-06,2.04e-07,Converged,1.24592494e-08,9.92e-06,2.05e-09,-7.80e-10,-6.67%
MINPERM,Converged,3.40283007e-04,8.76e-06,4.88e-06,Converged,3.49081269e-04,6.33e-06,2.98e-06,-8.80e-06,-2.59%
DNIEPER,MaxIter,1.87413505e+04,9.91e-06,1.06e-03,Converged,1.87440144e+04,9.84e-06,4.96e-06,-2.66e+00,-0.01%
HS85,MaxTime,-2.21565835e+00,7.67e-04,1.02e-03,Converged,-2.21560457e+00,9.53e-06,2.03e-06,-5.38e-05,-0.00%


In [11]:
statusses = base_df[['status']].join(new_df[['status']], lsuffix=' 0', rsuffix=' 1')
not_conv_to_conv = (statusses['status 0'] != 'Converged') & (statusses['status 1'] == 'Converged')
conv_to_not_conv = (statusses['status 0'] == 'Converged') & (statusses['status 1'] != 'Converged')

print(f'{len(base_df[not_conv_to_conv])} tests that didn\'t converge before do converge after the change')
print(f'{len(base_df[conv_to_not_conv])} tests that converged before no longer converge after the change')

display(HTML("<hr>"))

print('The following tests went from not converging to converging')
display(base_df[not_conv_to_conv].style.format('{:.2e}', subset=(base_df.dtypes == float)) \
                                       .format('{:.8e}', subset=['f']) \
                                       .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), base_df.columns))))
display(new_df[not_conv_to_conv].style.format('{:.2e}', subset=(new_df.dtypes == float)) \
                                      .format('{:.8e}', subset=['f']) \
                                      .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), new_df.columns))))

display(HTML("<hr>"))

print('The following tests went from converging to no longer converging')
display(base_df[conv_to_not_conv].style.format('{:.2e}', subset=(base_df.dtypes == float)) \
                                       .format('{:.8e}', subset=['f']) \
                                       .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), base_df.columns))))
display(new_df[conv_to_not_conv].style.format('{:.2e}', subset=(new_df.dtypes == float)) \
                                      .format('{:.8e}', subset=['f']) \
                                      .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), new_df.columns))))

6 tests that didn't converge before do converge after the change
6 tests that converged before no longer converge after the change


The following tests went from not converging to converging


,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,f evaluations,grad_f evaluations,g evaluations,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,,,
CONCON,MaxIter,1.54e+00,66108,240,49,7,20,-6.23079541e+03,6.64e-06,8.11e-05,607391,351848,607844,351848,29065,0,4,9.28e-02,2.76e+03,1.03e+00,15,11,5
DNIEPER,MaxIter,3.39e+00,80416,240,26,0,20,1.87413505e+04,9.91e-06,1.06e-03,397134,273122,397594,273122,9070,0,502,1.55e+03,1.93e+02,4.59e+03,61,24,120
EXPQUAD,MaxIter,8.56e-02,147,240,240,20,20,inf,inf,0.00e+00,660,774,901,774,0,0,0,0.00e+00,1.00e+20,0.00e+00,1200,0,200
HS85,MaxTime,3.11e+01,5768,10,5,0,5,-2.21565835e+00,7.67e-04,1.02e-03,339144,39952,339159,39952,3927,0,3971,2.07e+01,7.70e+02,6.23e-02,5,21,10
MCONCON,MaxIter,1.56e+00,59100,240,48,8,20,-6.23079563e+03,9.95e-06,6.77e-05,596433,338939,596885,338939,30176,0,1,5.57e-02,2.76e+03,1.03e+00,15,11,5
QCNEW,MaxIter,2.45e+00,219441,240,240,20,20,-8.04868295e+02,5.00e+03,0.00e+00,439122,439362,439562,439362,0,0,219420,6.33e-05,8.77e-01,0.00e+00,9,3,18


,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,f evaluations,grad_f evaluations,g evaluations,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,,,
CONCON,Converged,8.78e-01,68363,85,64,2,20,-6.23079555e+03,8.66e-06,9.05e-06,305961,285018,374387,285018,8424,0,1,1.19e+00,2.76e+03,1.03e+00,15,11,5
DNIEPER,Converged,1.89e+00,39730,45,36,0,20,1.87440144e+04,9.84e-06,4.96e-06,244908,198158,284663,198158,6947,0,0,7.83e+03,1.93e+02,4.59e+03,61,24,120
EXPQUAD,Converged,8.63e+00,21675,60,42,20,20,-3.68494055e+09,7.77e-06,0.00e+00,150970,86190,172665,86190,1064,0,21,0.00e+00,3.07e+04,0.00e+00,1200,0,200
HS85,Converged,2.96e+01,6378,15,6,0,6,-2.21560457e+00,9.53e-06,2.03e-06,195420,55007,201809,55007,4959,0,2395,7.19e+01,7.70e+02,6.09e-02,5,21,10
MCONCON,Converged,1.51e+00,59773,130,50,2,20,-6.23079556e+03,8.86e-06,9.93e-06,442118,330027,501999,330027,18400,0,0,1.19e+00,2.76e+03,1.03e+00,15,11,5
QCNEW,Converged,1.64e-03,17,1,0,0,0,-8.06521854e+02,0.00e+00,0.00e+00,154,111,172,111,1,0,14,1.73e+00,8.77e-01,0.00e+00,9,3,18


The following tests went from converging to no longer converging


,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,f evaluations,grad_f evaluations,g evaluations,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,,,
BRITGAS,Converged,2.82e+00,9329,20,5,0,5,0.00000000e+00,9.11e-06,7.54e-06,19106,18837,19141,18837,0,0,0,9.62e+01,6.05e+01,7.33e-05,450,360,474
ERRINBAR,Converged,1.83e+00,204157,208,203,2,20,2.80452569e+01,7.38e-06,7.50e-06,525836,463048,526230,463048,5934,0,21,5.29e+00,2.55e+03,6.55e-02,18,9,14
HOLMES,Converged,9.86e-01,141,6,0,0,0,1.24815035e+03,6.10e-06,0.00e+00,421,359,433,359,1,0,0,0.00e+00,7.75e-01,0.00e+00,180,0,180
HS103,Converged,1.33e-01,2479,13,2,1,1,5.43665352e+02,6.46e-06,1.54e-06,5622,5265,5646,5265,11,0,2,1.64e+05,6.40e+00,1.54e+03,7,5,14
HS105,Converged,4.55e-01,281,6,0,0,0,1.04461169e+03,8.97e-06,0.00e+00,1209,861,1221,861,37,0,2,1.00e+00,2.88e+02,0.00e+00,8,1,16
SMBANK,Converged,2.03e+00,37461,60,30,7,20,-7.12929200e+06,5.03e-06,2.65e-06,76147,75122,76240,75122,4,0,1,1.99e+00,2.11e+06,1.09e-01,117,64,234


,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,f evaluations,grad_f evaluations,g evaluations,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,,,
BRITGAS,MaxTime,3.02e+01,100162,104,97,0,20,0.00000000e+00,7.61e-04,2.57e-02,201791,300893,302037,300893,5,0,0,8.81e+01,7.06e+01,1.85e+01,450,360,474
ERRINBAR,MaxIter,2.02e+00,239124,240,238,1,20,2.83391434e+01,6.72e-02,2.34e-02,495272,724276,734615,724276,295,0,4,1.80e+00,2.39e+03,8.65e-02,18,9,14
HOLMES,MaxTime,3.01e+01,2548,196,196,20,20,1.28856061e+03,nan,0.00e+00,10388,10388,13092,10388,0,0,196,0.00e+00,0.00e+00,0.00e+00,180,0,180
HS103,MaxIter,3.14e+00,43359,240,109,4,20,1.95147480e+00,1.21e-02,3.00e+03,207540,187459,251115,187459,4259,15,129,2.90e-01,1.12e+01,2.04e+05,7,5,14
HS105,MaxTime,3.09e+01,16122,19,16,0,15,1.06186306e+03,1.54e+00,0.00e+00,245581,147737,261676,147737,14369,223,14117,1.00e+00,2.47e+02,0.00e+00,8,1,16
SMBANK,MaxIter,1.04e+01,211079,240,198,16,20,1.45906510e+07,4.69e-06,1.59e+06,354489,558002,565772,558002,81058,80852,161783,2.26e-03,4.93e+06,2.90e+03,117,64,234


In [12]:
both_converged = (base_df['status'] == 'Converged') & (new_df['status'] == 'Converged')
cmp = compare_results(base_df[both_converged], new_df[both_converged], ['status', 'time', 'f evaluations', 'grad_f evaluations'])
cmp['time imprv'] = cmp['time 0'] - cmp['time 1']
cmp['rel time imprv'] = 100 * cmp['time imprv'] / cmp['time 0']
cmp['f eval imprv'] = cmp['f evaluations 0'] - cmp['f evaluations 1']
cmp['rel f eval imprv'] = 100 * cmp['f eval imprv'] / cmp['f evaluations 0']
cmp['grad_f eval imprv'] = cmp['grad_f evaluations 0'] - cmp['grad_f evaluations 1']
cmp['rel grad_f eval imprv'] = 100 * cmp['grad_f eval imprv'] / cmp['grad_f evaluations 0']
print(f"Net time improvement:      {cmp['time imprv'].sum()}")
print(f"Relative time improvement: {100 * cmp['time imprv'].sum() / cmp['time 0'].sum():.02f}%")
print(f"Net f eval improvement:      {cmp['f eval imprv'].sum()}")
print(f"Relative f eval improvement: {100 * cmp['f eval imprv'].sum() / cmp['f evaluations 0'].sum():.02f}%")
print(f"Net grad_f eval improvement:      {cmp['grad_f eval imprv'].sum()}")
print(f"Relative grad_f eval improvement: {100 * cmp['grad_f eval imprv'].sum() / cmp['grad_f evaluations 0'].sum():.02f}%")
print('positive is good, negative is bad')


print(f"Time:                  {base_df[both_converged]['time'].sum()}\t{new_df[both_converged]['time'].sum()}")
print(f"Objective evaluations: {base_df[both_converged]['f evaluations'].sum()}\t{new_df[both_converged]['f evaluations'].sum()}")
print(f"Gradient evaluations:  {base_df[both_converged]['grad_f evaluations'].sum()}\t{new_df[both_converged]['grad_f evaluations'].sum()}")
print(f"Outer iterations:      {base_df[both_converged]['outer iterations'].sum()}\t{new_df[both_converged]['outer iterations'].sum()}")
print(f"Inner iterations:      {base_df[both_converged]['inner iterations'].sum()}\t{new_df[both_converged]['inner iterations'].sum()}")
print(f"Linesearch failures:   {base_df[both_converged]['linesearch failures'].sum()}\t{new_df[both_converged]['linesearch failures'].sum()}")
print(f"L-BFGS failures:       {base_df[both_converged]['L-BFGS failures'].sum()}\t{new_df[both_converged]['L-BFGS failures'].sum()}")
print(f"L-BFGS rejected:       {base_df[both_converged]['L-BFGS rejected'].sum()}\t{new_df[both_converged]['L-BFGS rejected'].sum()}")

cmp.sort_values('rel f eval imprv') \
   .style.applymap(color_negative_red_positive_green, subset=['time imprv', 'rel time imprv', 'f eval imprv', 'rel f eval imprv', 'grad_f eval imprv', 'rel grad_f eval imprv']) \
         .format('{:.2e}', subset=(cmp.dtypes == float)) \
         .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), cmp.columns)))

Net time improvement:      -28.581421999999996
Relative time improvement: -40.94%
Net f eval improvement:      -170295
Relative f eval improvement: -8.58%
Net grad_f eval improvement:      -830504
Relative grad_f eval improvement: -51.92%
positive is good, negative is bad
Time:                  69.81917700000001	98.40059900000001
Objective evaluations: 1985514	2155809
Gradient evaluations:  1599513	2430017
Outer iterations:      1694	1915
Inner iterations:      635529	697142
Linesearch failures:   32514	37549
L-BFGS failures:       0	2975
L-BFGS rejected:       23571	30730


,status 0,time 0,f evaluations 0,grad_f evaluations 0,status 1,time 1,f evaluations 1,grad_f evaluations 1,time imprv,rel time imprv,f eval imprv,rel f eval imprv,grad_f eval imprv,rel grad_f eval imprv
name,,,,,,,,,,,,,,
HIMMELP4,Converged,1.48e-04,15,18,Converged,3.30e-03,3401,2737,-3.15e-03,-2131.76%,-3386,-22573.33%,-2719,-15105.56%
HIMMELP3,Converged,8.50e-05,15,18,Converged,4.66e-03,1443,1571,-4.58e-03,-5387.06%,-1428,-9520.00%,-1553,-8627.78%
HS112,Converged,7.85e-03,489,476,Converged,3.22e-01,32203,46198,-3.14e-01,-3996.34%,-31714,-6485.48%,-45722,-9605.46%
DECONVC,Converged,1.94e-01,3200,2780,Converged,3.18e+00,52754,72783,-2.99e+00,-1542.97%,-49554,-1548.56%,-70003,-2518.09%
PFIT3,Converged,9.07e-03,1920,1094,Converged,4.12e-02,23097,11025,-3.21e-02,-353.93%,-21177,-1102.97%,-9931,-907.77%
FEEDLOC,Converged,2.97e-01,8001,6770,Converged,3.74e+00,74797,88041,-3.45e+00,-1158.88%,-66796,-834.85%,-81271,-1200.46%
EQC,Converged,2.31e-01,42024,42046,Converged,1.11e+00,356709,209878,-8.75e-01,-378.38%,-314685,-748.82%,-167832,-399.16%
DECONVBNE,Converged,2.88e-01,7998,7359,Converged,1.78e+00,62150,84269,-1.49e+00,-516.99%,-54152,-677.07%,-76910,-1045.11%
QC,Converged,9.62e-04,241,159,Converged,1.14e-02,1631,964,-1.04e-02,-1080.15%,-1390,-576.76%,-805,-506.29%


In [13]:
cmp = compare_results(base_df[both_converged], new_df[both_converged], ['status', 'linesearch failures', 'L-BFGS failures', 'L-BFGS rejected'])
cmp['ls imprv'] = cmp['linesearch failures 0'] - cmp['linesearch failures 1']
cmp['rel ls imprv'] = 100 * cmp['ls imprv'] / cmp['linesearch failures 0']
cmp['lbfgs imprv'] = cmp['L-BFGS failures 0'] - cmp['L-BFGS failures 1']
cmp['rel lbfgs imprv'] = 100 * cmp['lbfgs imprv'] / cmp['L-BFGS failures 0']
cmp['lbfgs rej imprv'] = cmp['L-BFGS rejected 0'] - cmp['L-BFGS rejected 1']
cmp['rel lbfgs rej imprv'] = 100 * cmp['lbfgs rej imprv'] / cmp['L-BFGS rejected 0']
print(f"Net linesearch failures improvement:    {cmp['ls imprv'].sum()}")
print(f"Relative linesearch improvement:        {100. * cmp['ls imprv'].sum() / cmp['linesearch failures 0'].sum():.02f}%")
print(f"Net L-BFGS failures improvement:        {cmp['lbfgs imprv'].sum()}")
print(f"Relative L-BFGS failures improvement:   {100. * cmp['lbfgs imprv'].sum() / cmp['L-BFGS failures 0'].sum():.02f}%")
print(f"Net L-BFGS rejections improvement:      {cmp['lbfgs rej imprv'].sum()}")
print(f"Relative L-BFGS rejections improvement: {100. * cmp['lbfgs rej imprv'].sum() / cmp['L-BFGS rejected 0'].sum():.02f}%")

cmp.sort_values('rel ls imprv') \
   .style.applymap(color_negative_red_positive_green, subset=['ls imprv', 'rel ls imprv', 'lbfgs imprv', 'rel lbfgs imprv', 'lbfgs rej imprv', 'rel lbfgs rej imprv']) \
         .format('{:.2e}', subset=(cmp.dtypes == float)) \
         .format('{:.02f}%', subset=list(filter(lambda s: s.startswith('rel'), cmp.columns)))

Net linesearch failures improvement:    -5035
Relative linesearch improvement:        -15.49%
Net L-BFGS failures improvement:        -2975
Relative L-BFGS failures improvement:   -inf%
Net L-BFGS rejections improvement:      -7159
Relative L-BFGS rejections improvement: -30.37%
<ipython-input-13-d8e974567ec9>:11: RuntimeWarning: divide by zero encountered in double_scalars
  print(f"Relative L-BFGS failures improvement:   {100. * cmp['lbfgs imprv'].sum() / cmp['L-BFGS failures 0'].sum():.02f}%")


,status 0,linesearch failures 0,L-BFGS failures 0,L-BFGS rejected 0,status 1,linesearch failures 1,L-BFGS failures 1,L-BFGS rejected 1,ls imprv,rel ls imprv,lbfgs imprv,rel lbfgs imprv,lbfgs rej imprv,rel lbfgs rej imprv
name,,,,,,,,,,,,,,
HIMMELP3,Converged,0,0,3,Converged,39,7,317,-39,-inf%,-7,-inf%,-314,-10466.67%
MINC44,Converged,0,0,0,Converged,16,0,0,-16,-inf%,0,nan%,0,nan%
HS62,Converged,0,0,0,Converged,2,0,0,-2,-inf%,0,nan%,0,nan%
HS2,Converged,0,0,0,Converged,4,0,0,-4,-inf%,0,nan%,0,nan%
HS17,Converged,0,0,0,Converged,10,10,0,-10,-inf%,-10,-inf%,0,nan%
HS16,Converged,0,0,0,Converged,10,10,1,-10,-inf%,-10,-inf%,-1,-inf%
HS112,Converged,0,0,0,Converged,2949,2939,5927,-2949,-inf%,-2939,-inf%,-5927,-inf%
HIMMELP4,Converged,0,0,3,Converged,151,7,317,-151,-inf%,-7,-inf%,-314,-10466.67%
WATER,Converged,0,0,0,Converged,3,0,0,-3,-inf%,0,nan%,0,nan%


In [14]:
base_df[base_df['ε'] == 0]

,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,...,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,
EQC,Converged,0.231330,21001,22,21,20,0,-829.547705,0.0,0.00,...,42046,0,0,20979,6.332652e-05,0.876608,0.000000e+00,9,3,18
HIMMELP3,Converged,0.000085,5,3,0,0,0,-59.013178,0.0,0.00,...,18,0,0,3,1.414214e+00,99.247166,0.000000e+00,2,2,4
HIMMELP4,Converged,0.000148,5,3,0,0,0,-59.013178,0.0,0.00,...,18,0,0,3,1.732051e+00,99.247166,0.000000e+00,2,3,4
HIMMELP5,Converged,0.000113,8,2,0,0,0,-59.013178,0.0,0.00,...,23,0,0,0,1.732051e+00,99.247166,0.000000e+00,2,3,4
HIMMELP6,Converged,0.000106,8,2,0,0,0,-59.013178,0.0,0.00,...,23,0,0,0,2.236068e+00,99.247166,0.000000e+00,2,5,4
HS30,Converged,0.000141,3,2,0,0,0,1.000000,0.0,0.00,...,10,0,0,0,1.000000e+00,1.000000,0.000000e+00,3,1,6
HS4,Converged,0.000025,0,2,0,0,0,2.666667,0.0,0.00,...,6,0,0,0,0.000000e+00,1.000000,0.000000e+00,2,0,2
HS45,Converged,0.000100,5,3,0,0,0,1.000000,0.0,0.00,...,20,0,0,2,0.000000e+00,7.416198,0.000000e+00,5,0,10
HS93,MaxIter,0.001358,25,240,0,0,0,0.000000,0.0,2.07,...,792,0,0,0,1.000000e+09,13.752958,3.070000e+09,6,2,6


In [15]:
new_df[new_df['ε'] == 0]

,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,...,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,
EQC,Converged,1.106631,21001,22,21,20,0,-829.547705,0.0,0.00,...,209878,20979,0,21001,6.332652e-05,0.876608,0.000000e+00,9,3,18
HIMMELP3,Converged,0.004664,428,24,21,0,20,-59.013178,0.0,0.00,...,1571,39,7,317,1.414214e+00,99.247166,0.000000e+00,2,2,4
HIMMELP4,Converged,0.003303,549,24,21,0,20,-59.013178,0.0,0.00,...,2737,151,7,317,1.732051e+00,99.247166,0.000000e+00,2,3,4
HIMMELP5,Converged,0.000662,8,2,0,0,0,-59.013178,0.0,0.00,...,29,0,0,0,1.732051e+00,99.247166,0.000000e+00,2,3,4
HIMMELP6,Converged,0.001083,8,2,0,0,0,-59.013178,0.0,0.00,...,29,0,0,0,2.236068e+00,99.247166,0.000000e+00,2,5,4
HS21MOD,Converged,0.000920,3,1,0,0,0,-95.960000,0.0,0.00,...,10,0,0,0,1.000000e+00,2.828427,0.000000e+00,7,1,8
HS4,Converged,0.000554,0,2,0,0,0,2.666667,0.0,0.00,...,6,0,0,0,0.000000e+00,1.000000,0.000000e+00,2,0,2
HS45,Converged,0.000095,11,2,0,0,0,1.000000,0.0,0.00,...,36,0,0,11,0.000000e+00,7.416198,0.000000e+00,5,0,10
HS93,MaxIter,0.001907,10,240,0,0,0,0.000000,0.0,2.07,...,761,1,0,0,1.000000e+09,13.358319,3.070000e+09,6,2,6


In [16]:
print('"' + '"\n"'.join(new_df[new_df['box constr x'] > 0].index.values) + '"')

"3PK"
"A4X12"
"ACOPP14"
"ACOPP30"
"ACOPR14"
"AIRCRFTA"
"AIRPORT"
"ALLINITA"
"ALLINITC"
"ALSOTAME"
"ANTWERP"
"AVION2"
"BATCH"
"BRITGAS"
"BT13"
"C-RELOAD"
"CANTILVR"
"CONCON"
"CORE2"
"CRESC50"
"DALLASM"
"DALLASS"
"DECONVBNE"
"DECONVC"
"DECONVNE"
"DECONVU"
"DEGENLPA"
"DEGENLPB"
"DEMBO7"
"DISC2"
"DISCS"
"DNIEPER"
"EQC"
"ERRINBAR"
"EXPLIN"
"EXPLIN2"
"EXPQUAD"
"FCCU"
"FEEDLOC"
"FLETCHER"
"FLOSP2TM"
"GAUSSELM"
"GILBERT"
"GRIDGENA"
"GRIDNETA"
"GRIDNETI"
"GROUPING"
"HADAMARD"
"HANGING"
"HELSBY"
"HIMMELBI"
"HIMMELBJ"
"HIMMELBK"
"HIMMELP1"
"HIMMELP2"
"HIMMELP3"
"HIMMELP4"
"HIMMELP5"
"HIMMELP6"
"HOLMES"
"HONG"
"HS101"
"HS102"
"HS103"
"HS104"
"HS105"
"HS106"
"HS107"
"HS108"
"HS109"
"HS110"
"HS111"
"HS112"
"HS114"
"HS116"
"HS117"
"HS119"
"HS13"
"HS15"
"HS16"
"HS17"
"HS18"
"HS19"
"HS2"
"HS20"
"HS21MOD"
"HS23"
"HS24"
"HS25"
"HS25NE"
"HS2NE"
"HS30"
"HS31"
"HS32"
"HS33"
"HS34"
"HS36"
"HS37"
"HS38"
"HS4"
"HS41"
"HS45"
"HS5"
"HS54"
"HS55"
"HS57"
"HS59"
"HS60"
"HS62"
"HS63"
"HS64"
"HS65"
"HS66"
"HS67"
"HS6

In [17]:
new_df[new_df.index == 'HS75']

,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,...,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,
HS75,Converged,0.006897,706,10,0,0,0,5174.394702,0.000006,0.000007,...,2287,9,0,0,974454.492886,1207.64872,2778.524901,4,5,8


In [18]:
base_df[base_df.index == 'HS75']

,status,time,inner iterations,outer iterations,inner convergence failures,initial penalty reduced,penalty reduced,f,ε,δ,...,grad_g evaluations,linesearch failures,L-BFGS failures,L-BFGS rejected,‖Σ‖,‖x‖,‖y‖,n,m,box constr x
name,,,,,,,,,,,,,,,,,,,,,
HS75,Converged,0.007224,594,10,0,0,0,5174.394702,0.000007,0.000007,...,1263,6,0,0,974454.917815,1207.64872,2778.524902,4,5,8
